In [8]:
import pyhepmc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
particle_info = pd.read_csv("genie_tau_decays_particles.csv")

In [10]:
particle_info

,event_num,pdg,E,px,py,pz
0,0,1000080160,14.895081,0.000000,0.000000,0.000000
1,0,-16,3.730323,0.185260,3.295873,1.737299
2,0,-15,3.251366,-0.023894,2.329548,1.409583
3,0,211,0.726490,-0.435732,-0.013640,0.564145
4,0,-16,2.524877,0.411767,2.343313,0.845184
...,...,...,...,...,...,...
447119,76642,16,2206.526149,-1049.708315,-1364.091472,-1380.624698
447120,76642,15,2128.943214,-1013.673964,-1313.010045,-1334.490828
447121,76642,13,672.544885,-319.716273,-414.716891,-422.028455
447122,76642,-14,778.763440,-371.325492,-480.035241,-488.012338


In [15]:
event = particle_info.query("event_num == 0")
event

,event_num,pdg,E,px,py,pz
0,0,1000080160,14.895081,0.000000,0.000000,0.000000
1,0,-16,3.730323,0.185260,3.295873,1.737299
2,0,-15,3.251366,-0.023894,2.329548,1.409583
3,0,211,0.726490,-0.435732,-0.013640,0.564145
4,0,-16,2.524877,0.411767,2.343313,0.845184


In [54]:
assert (particle_info
        .groupby("event_num")
        .filter(
            lambda df: 
                (set(np.abs(df.iloc[3:, 1]).tolist()) == {12, 16})
                  and (df.shape[0] > 5)
        )
        .shape[0] == 0)

In [77]:
event0 = particle_info.query("event_num == 0")

def csv2genevent(event: pd.DataFrame) -> pyhepmc.GenEvent:
    evt = pyhepmc.GenEvent(pyhepmc.Units.GEV, pyhepmc.Units.MM)
    
    # Extract particles        
    nucleus = pyhepmc.GenParticle(pyhepmc.FourVector(*event.iloc[0, 3:], event.iloc[0, 2]), event.iloc[0, 1], 3)
    nutau = pyhepmc.GenParticle(pyhepmc.FourVector(*event.iloc[1, 3:], event.iloc[1, 2]), event.iloc[1, 1], 3)
    tau = pyhepmc.GenParticle(pyhepmc.FourVector(*event.iloc[2, 3:], event.iloc[2, 2]), event.iloc[2, 1], 3)
    decay_products = [pyhepmc.GenParticle(pyhepmc.FourVector(*event.iloc[i, 3:], event.iloc[i, 2]), event.iloc[i, 1], 1) for i in range(3, len(event))]

    # The electron is omitted due to simulation optimizations when the tau decays to an electron and a neutrino.
    # We add it back by assuming 4-momentum conservation when the tau decays.
    if set(np.abs(event.iloc[3:, 1]).tolist()) == {12, 16}:
        electron_4m = tau.momentum - sum([p.momentum for p in decay_products], pyhepmc.FourVector())

        decay_pdgs = event.iloc[3:, 1].values
        if 12 in decay_pdgs:
            electron_pdg = -11
        elif -12 in decay_pdgs:
            electron_pdg = 11
        else:
            raise ValueError(f"No valid electron neutrino found. PDG IDs are {decay_pdgs.tolist()}. Event number: {event.iloc[0, 0]}.")
    
        decay_products.append(pyhepmc.GenParticle(electron_4m, electron_pdg, 1))

    # Create vertices
    interaction_vertex = pyhepmc.GenVertex()
    interaction_vertex.add_particle_in(nucleus)
    interaction_vertex.add_particle_in(nutau)
    interaction_vertex.add_particle_out(tau)

    tau_decay_vertex = pyhepmc.GenVertex()
    tau_decay_vertex.add_particle_in(tau)
    for p in decay_products:
        tau_decay_vertex.add_particle_out(p)

    evt.add_vertex(interaction_vertex)
    evt.add_vertex(tau_decay_vertex)
    return evt

csv2genevent(event0)

<GenEvent momentum_unit=1, length_unit=0, event_number=0, particles=5, vertices=2, run_info=None>

In [71]:
event2 = particle_info.query("event_num == 2")

In [76]:
-12 in event2.iloc[3:, 1].values

True

In [80]:
all_events = []

with pyhepmc.open("test.dat", "w") as f:
    for event_num, event in particle_info.groupby("event_num"):
        evt = csv2genevent(event)
        evt.event_number = event_num
        all_events.append(evt)
        
        if event_num > 10:
            break

    f.write(evt)

In [82]:
all_events[2]

<GenEvent momentum_unit=1, length_unit=0, event_number=2, particles=6, vertices=2, run_info=None>